In [11]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D,concatenate, Conv2DTranspose
from tensorflow.keras.models import Model


In [12]:
def conv_block(x, n_filters):
    '''Two convolutional layers'''
    x = Conv2D(n_filters, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(n_filters, (3, 3), padding='same', activation='relu')(x)
    return x

def encoder_block(x, n_filters):
    '''Convolution block and max pooling'''
    x = conv_block(x, n_filters)
    p = MaxPooling2D((2, 2))(x)
    return x, p  # Skip connection for later use in the decoder

def decoder_block(x, skip_features, n_filters):
    '''Upsampling and concatenation'''
    x = Conv2DTranspose(n_filters, (2, 2), strides=2, padding='same')(x)
    x = concatenate([x, skip_features])
    x = conv_block(x, n_filters)
    return x

def unet_model(n_classes, img_height, img_width, img_channels):
    inputs = Input((img_height, img_width, img_channels))

    # Encoder
    c1, p1 = encoder_block(inputs, n_filters=64)
    c2, p2 = encoder_block(p1, n_filters=128)
    c3, p3 = encoder_block(p2, n_filters=256)
    c4, p4 = encoder_block(p3, n_filters=512)

    # Bottleneck
    bridge = conv_block(p4, n_filters=1024)

    # Decoder
    u4 = decoder_block(bridge, c4, n_filters=512)
    u3 = decoder_block(u4, c3, n_filters=256)
    u2 = decoder_block(u3, c2, n_filters=128)
    u1 = decoder_block(u2, c1, n_filters=64)

    outputs = Conv2D(n_classes, (1, 1), padding='same', activation='softmax')(u1)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Model parameters
img_height = 128
img_width = 128
img_channels = 3
n_classes = 6

# Build the model
model = unet_model(n_classes, img_height, img_width, img_channels)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 128, 128, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_38 (Conv2D)        │ (None, 128, 128, 64)   │          1,792 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_39 (Conv2D)        │ (None, 128, 128, 64)   │         36,928 │ conv2d_38[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_8           │ (None, 64, 64, 64)     │              0 │ conv2d_39[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_40 (Conv2D)        │ (None, 64, 64, 128)    │         73,856 │ max_pooling2d_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_41 (Conv2D)        │ (None, 64, 64, 128)    │        147,584 │ conv2d_40[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_9           │ (None, 32, 32, 128)    │              0 │ conv2d_41[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_42 (Conv2D)        │ (None, 32, 32, 256)    │        295,168 │ max_pooling2d_9[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_43 (Conv2D)        │ (None, 32, 32, 256)    │        590,080 │ conv2d_42[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_10          │ (None, 16, 16, 256)    │              0 │ conv2d_43[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_44 (Conv2D)        │ (None, 16, 16, 512)    │      1,180,160 │ max_pooling2d_10[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_45 (Conv2D)        │ (None, 16, 16, 512)    │      2,359,808 │ conv2d_44[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_11          │ (None, 8, 8, 512)      │              0 │ conv2d_45[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_46 (Conv2D)        │ (None, 8, 8, 1024)     │      4,719,616 │ max_pooling2d_11[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_47 (Conv2D)        │ (None, 8, 8, 1024)     │      9,438,208 │ conv2d_46[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_8        │ (None, 16, 16, 512)    │      2,097,664 │ conv2d_47[0][0]        │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_8        

 Total params: 31,032,070 (118.38 MB)

 Trainable params: 31,032,070 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile(loss="categorical_crossentropy", categorical_crossentropy)

SyntaxError: positional argument follows keyword argument (<ipython-input-13-e926d934bd02>, line 4)